In [1]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab05.ok')

Assignment: lab05
OK, version v1.13.11



# Lab 5

In this lab you will have a chance to try using Pytorch to fit a basic model to a real-world dataset.  Once you have completed this lab you will have:

1. Loaded and cleaned a real-world fixed-width data file.
2. Fit a linear model using the closed form equations from lecture.
3. Constructed and fit a model using Pytorch.

**This assignment should be completed and submitted by 11:59 PM on Saturday March 7, 2020.**

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** in the following cell:


*List collaborators here*

## Libraries Used

In this lab you will get a chance to use a number of new software libraries.

In [2]:
import pandas as pd
import numpy as np

The following torch libraries are used to construct and train models via gradient descent.

1. PyTorch (`torch`) is the main pytorch library.
2. `torch.nn` is the model design sub-library (`nn` stands for neural network.) 
3. `torch.nn.functional` contains commonly used functions (e.g., `mse_loss` and `abs_loss`) when building models.
4. The `TensorDataset` and `DataLoader` libraries are used to load data efficiently and simplify data sampling.

If you are interested in learning more about fitting neural networks and using PyTorch checkout this tutorial: [PyTorch in 60 Minutes](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).  It is worth noting that the basic modeling concepts we teach in this class are also part of this tutorial.


In [3]:
import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In this lab you will have a chance to try using Plotly.  
1. `plotly.offline` is the core plotly library and is mainly used for the `py.iplot` syntax. 
2. `plotly.express` contains very simple `matplotlib` like functions for making standard plots
3. `plotly.graph_objects` contains basic plot elements (e.g., `Scatter`, `Lines`, `Bars`, `Contours`) that can be assembled to build plots. 
4. `cufflinks` is a library that actually modifies the behavior of pandas to allow `df.iplot` mimic the behavior of `df.plot` but instead using `plotly` plots.

In [4]:
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
cf.set_config_file(offline=True, sharing=False, theme='ggplot')

A minor note about **Plotly and JupyterLab**.  The latest version of Plotly requires JupyterLab extensions to run in the JupyterLab environment.  Unfortunately, we have not installed those extensions on the DataHub.  You can modify your local environment to work but in general you may need to use the classic notebook environment to use the Plotly plots.  

<br/><br/><br/>

---
<br/><br/><br/>

## Part 0: Loading the Data

For this lab, will use data obtained from the [Earth Systems Research Laboratory (NOAA)](https://www.esrl.noaa.gov/gmd/ccgg/trends/) website. This data was collected at the Mauna Loa Observatory in Hawaii.  We have already downloaded the data into file:

```
data = "data/c02.txt"
```

Read the documentation attached to the file and examine how it is organized.


In [5]:
def print_head(filename, n=10):
    """
    Print all the line number, a tab character, and the line for all the 
    lines that begin with the comment_char.
    """
    with open(filename, "r") as f:
        for i in range(n):
            print(i, "\t", f.readline().strip())

In [6]:
print_head("data/co2.txt", 100)

0 	 # --------------------------------------------------------------------
1 	 # USE OF NOAA ESRL DATA
2 	 #
3 	 # These data are made freely available to the public and the
4 	 # scientific community in the belief that their wide dissemination
5 	 # will lead to greater understanding and new scientific insights.
6 	 # The availability of these data does not constitute publication
7 	 # of the data.  NOAA relies on the ethics and integrity of the user to
8 	 # ensure that ESRL receives fair credit for their work.  If the data
9 	 # are obtained for potential use in a publication or presentation,
10 	 # ESRL should be informed at the outset of the nature of this work.
11 	 # If the ESRL data are essential to the work, or if an important
12 	 # result or conclusion depends on the ESRL data, co-authorship
13 	 # may be appropriate.  This should be discussed at an early stage in
14 	 # the work.  Manuscripts using the ESRL data should be sent to ESRL
15 	 # for review before they are submi

This is a fixed width file format.  The following Pandas code will load the file into a DataFrame.  Take a look at the output.

In [7]:
column_names=["Year", "Month", "Date", "Average", 
              "Interpolated", "Trend", "Num Days"]

data = pd.read_fwf("data/co2.txt", comment="#", names=column_names) 

data.head()

,Year,Month,Date,Average,Interpolated,Trend,Num Days
0,1958,3,1958.208,315.71,315.71,314.62,-1
1,1958,4,1958.292,317.45,317.45,315.29,-1
2,1958,5,1958.375,317.50,317.50,314.71,-1
3,1958,6,1958.458,-99.99,317.10,314.85,-1
4,1958,7,1958.542,315.86,315.86,314.98,-1


Run the following command to plot the data.  You will notice some missing values have been coded as -99.99.  

In [8]:
data.iplot(kind='line', x="Date", y="Average", 
           xTitle="Year", yTitle="CO2 (PPM)")

Let's remove these datapoints from the data.  

In [9]:
data = data[data["Average"] != -99.99] 

The following visualization should no longer have negative outliers.  

In [10]:
data.iplot(kind='scatter', x="Date", y="Average", 
           xTitle="Year", yTitle="CO2 (PPM)")

<br/><br/><br/>

---

<br/><br/>

## Part 1

In this part of the assignment you will fit the above data using one dimensional least squares regression introduced in lecture. In particular, given two data vectors $x$ and $y$ of length $n$, we would like to find the slope $m$ and intercept $b$ that minimizes 
$$
L(m, b) = \sum_{i=1}^{n} \left(y_i - \left(m x_i +b\right) \right)^2
$$

<br/><br/><br/>

---

### Question 1

Using the implementation of `standard_units` below, implement the remaining three functions. Note that the **correlation** between two vectors $x$ and $y$ of length $n$ is defined as $\frac{1}{n} \sum_{i=1}^{n} x_i y_i$.  You will find the answer to these questions in lecture but try to answer them yourself first.  Also **don't use for loops**.  

In [13]:
# We are giving you this function because we have written it in a way
# that will also work with Pytorch vectors. (Do you see why?)
def standard_units(x):
    """Return the 1d vector in standard unit form."""
    return (x - x.mean()) / x.std() 

<!--
BEGIN QUESTION
name: q1a
-->

In [16]:
def correlation(x, y):
    """Compute the correlation between two 1d vectors."""
    return np.mean(standard_units(x)*standard_units(y))

In [17]:
ok.grade("q1a");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<!--
BEGIN QUESTION
name: q1b
-->

In [20]:
def slope(x, y):
    """Compute the slope of the least squares regression line."""
    return correlation(x, y)*(np.std(y)/np.std(x))

In [21]:
ok.grade("q1b");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<!--
BEGIN QUESTION
name: q1c
-->

In [22]:
def intercept(x, y):
    """Compute the intercept of the least squares regression line."""
    return np.mean(y)-slope(x,y)*np.mean(x)

In [23]:
ok.grade("q1c");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



If you implemented things correctly, the following will print the slope and intercept of your model.

In [24]:
m = slope(data["Date"], data["Average"])
b = intercept(data["Date"], data["Average"])

# Make a prediction
yhat = m * data['Date'] + b
residual = data['Average'] - yhat
print("Slope:", m)
print("Intercept:", b)

Slope: 1.5705357679342555
Intercept: -2769.1558858518997


Notice that the interecept has a large negative value.  This will be an issue in later steps.

#### Visualizing the Model

Run the following to visualize the line of best fit and the corresponding residual.

In [26]:
# Visualize predictions
fig = make_subplots(rows=2, cols=1, shared_xaxes=True) 

# Add the Data
fig.add_trace(go.Scatter(x=data["Date"], y=data["Average"], name="Data",
                         mode="markers+lines", 
                         marker=go.scatter.Marker(size=2.5),
                         line=go.scatter.Line(width=1)), row=1, col=1)
# Add the line of best fit.
fig.add_trace(go.Scatter(x=data["Date"], y=yhat, name='Best Fit Line' ),
              row=1, col=1)

# Plot the Residual 
fig.add_trace(go.Scatter(x=data["Date"], y=residual, name='Residuals' ),
              row=2, col=1)

# Axis Labels
fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=700)

Do you notice any pattern in the residual?  Is there any structure left in the relationship between $y$ and $x$?  The next part of this problem will try to address this structure.

<br/><br/><br/>

---

<br/><br/>


## Part 2

Notice in the above plot that there is some parabolic curvature in the residual.  In this part of the assignment, we will improve our model by adding a quadratic component using PyTorch. Adding a quadratic component to the model makes it difficult but not impossible to find an analytic solution to the regression problem.  In future lectures we will see that there is indeed an analytic solution to this linear model.  However, here we will resort to using numerical iterative methods and in particular SGD to find the optimal model.  PyTorch comes in handy here because given the model and the loss function, PyTorch can compute the gradient that is needed for SGD.


We first convert the data into Pytorch tensors in the following lines.

In [27]:
x = torch.from_numpy(data[['Date']].to_numpy())
y = torch.from_numpy(data[['Average']].to_numpy())
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: torch.Size([736, 1])
Shape of y: torch.Size([736, 1])


<br/><br/><br/>

---

### Question 2

Given that the above data has some parabolic curvature, we would like to try and fit a model of the form:

$$
y = w_0 + w_1 x + w_2 x^2
$$

Finish implementing the `forward` function for our model in PyTorch below:

<!--
BEGIN QUESTION
name: q2
-->

In [28]:
class QuadraticModel(nn.Module):
    def __init__(self):
        super().__init__()
        # We initialized three weights [0,0,0] for our model.
        self.w = nn.Parameter(torch.zeros(3,1))
    def forward(self, x):
        w = self.w # you can access w[0], w[1], and w[2]
        y = w[0] + w[1]*x + w[2]*x*x
        return y

In [29]:
ok.grade("q2");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<br/><br/><br/>

---


### Question 3

Fill in the implementation for the mean squared loss function below. Please make sure that your implementation is compatible with PyTorch. Considering using `torch.mean`.

<!--
BEGIN QUESTION
name: q3
-->

In [41]:
def mse_loss(y, yhat):
    """Compute the mean squared loss between the tensors y and yhat"""
    return torch.mean((yhat-y)**2)

In [42]:
ok.grade("q3");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



Now we can compute the mean squared error of our initial model with parameters initialized as all zeros on our dataset with the following code.

In [43]:
m = QuadraticModel()
mse_loss(y, m(x)).item()

126944.18308288044

<br/><br/><br/>

---


### Question 4

Now that you have implemented the model and the loss, please implement the following SGD function.  Note that the solution is provided in lecture but try to solve the problem yourself first.

<!--
BEGIN QUESTION
name: q4
points: 0
-->

In [60]:
def sgd(model, loss_fn, dataset, lr=lambda t: 1./(1.+t), nepochs=10, batch_size=10):
    """
    Run SGD on the provdied model, loss function, and pytorch dataset.
    """
    loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
    for t in range(nepochs):
        for (x, y) in loader:
            # Steps:
            # 1. Compute the loss
            # 2. Run backpropagation on the loss.
            # 3. With torch.no_grad():
            #     a) for each parameter p in model.parameters()
            #         i. subtract the learning rate lr(t) times gradient (p.grad)
            #.    b) Zero the gradient after you used it. (model.zero_grad())
            loss = loss_fn(model(x), y)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= lr(t) * p.grad
                model.zero_grad()

**Note:** In contrast to lecture here we have modified the learning rate to be a function of the epoch rather than a constant.  This allows you to experiment with different learning rate schedules.  You might consider:

```
lr = lambda t: 0.1
lr = lambda t: 1.0/np.sqrt(1.0 + t)
lr = lambda t: 1.0/(1.0 + t)
```

<br/><br/><br/>

---

### Question 5

Let's test your implementation of PyTorch model, loss, and SGD.  Here we will create a tiny toy dataset from a toy model.


In [61]:
toy_model = QuadraticModel()
toy_model.w.data = torch.tensor([0.5, -0.2, 1.])
toy_x = torch.linspace(-1,1,50)
toy_y = toy_model(toy_x).detach()

In [62]:
px.scatter(x=toy_x.numpy(), y=toy_y.numpy())

Try running SGD on this toy data by picking some reasonable values for the learning rate `lr`, the `batch_size`, and the number of epochs `nepochs`.  

Suggestions: 
1. Keep the batch size small relative to the dataset size.
2. Try the default learning rate function and perhaps a constant function `lr = lambda t: 0.1`.  You might also try `lr = lambda t: 1./ np.sqrt(t + 1)`

<!--
BEGIN QUESTION
name: q5
points: 2
-->

In [71]:
batch_size = 10000
nepochs = 20
lr = lambda t: 1./ np.sqrt(t + 1)

model = QuadraticModel() 
dataset = TensorDataset(toy_x, toy_y)
sgd(model, mse_loss, dataset, lr=lr, batch_size=batch_size, nepochs=nepochs)

yhat = model(toy_x).detach()
print("Loss:", mse_loss(toy_y, yhat).item())
print("Weights:", model.w)

Loss: 0.003299491945654154
Weights: Parameter containing:
tensor([[ 0.5699],
        [-0.1997],
        [ 0.8158]], requires_grad=True)


In [72]:
ok.grade("q5");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



Did we estimate the model correctly?  Let's visualize the predictions.

In [73]:
fig = px.scatter(x=toy_x.numpy(), y=toy_y.numpy())
# Add the line of best fit.
fig.add_trace(go.Scatter(x=toy_x.numpy(), y=yhat.numpy(), name='Best Fit Line' ))
# Axis Labels
fig.update_xaxes(title_text="X")
fig.update_yaxes(title_text="Y")

<br/><br/><br/>

---

### Question 6

Try running SGD for a few epochs on our CO2 data.

In [74]:
batch_size = 100 
nepochs = 1
lr = lambda t: 1./ np.sqrt(t + 1) 

model = QuadraticModel() 
dataset = TensorDataset(x, y)
sgd(model, mse_loss, dataset, lr=lr, batch_size=batch_size, nepochs=nepochs)

yhat = model(x).detach()
print("Loss:", mse_loss(y, yhat).item())
print("Weights:", model.w)

Loss: nan
Weights: Parameter containing:
tensor([[nan],
        [nan],
        [nan]], requires_grad=True)


If the output above is a lot of `nan` you are not alone.  What went wrong?  Our initial model parameters $[0, 0, 0]$ are really far from optimum.  Recall from before that the intercept of the line was a very large negative number.  There are a few solutions. 

1. We could try to adjust our learning rate and using clipping to to prevent diverging gradients.  
2. We could standardize our data so there are no large values.  (We will do this one). 

Use the standardize function from before to transform the data into standard units.

<!--
BEGIN QUESTION
name: q6
-->

In [75]:
xs = standard_units(x)
ys = standard_units(y)

In [76]:
ok.grade("q6");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



<br/><br/><br/>

---

### Question 7

Try running the algorithm once more on our standardized data by picking some reasonable values for the learning rate `lr`, the `batch_size`, and the number of epochs `nepochs`.  

<!--
BEGIN QUESTION
name: q7
-->

In [107]:
batch_size = 1000
nepochs = 20
lr = lambda t: 1./ np.sqrt(t + 1)
dataset = TensorDataset(xs, ys)

model = QuadraticModel() 
sgd(model, mse_loss, dataset, lr=lr, batch_size=batch_size, nepochs=nepochs)

yhat = model(xs).detach() 
residual = (ys - yhat).detach().flatten()
print("Loss:", mse_loss(ys, yhat).item())
print("Weights:", model.w)

Loss: 0.0061124889875006925
Weights: Parameter containing:
tensor([[-0.1431],
        [ 0.9900],
        [ 0.1434]], requires_grad=True)


In [108]:
ok.grade("q7");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



In [109]:
# Visualize predictions
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=xs.flatten(), y=ys.flatten(), name="Data",
                         mode="markers+lines", 
                         marker=go.scatter.Marker(size=2.5),
                         line=go.scatter.Line(width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=xs.flatten(), y=yhat.detach().flatten(), 
                         name='Best Quadratic' ),
              row=1, col=1)
# Bottom Plot
fig.add_trace(go.Scatter(x=xs.flatten(), y=residual, name='Residuals'),
              row=2, col=1)
# Axis Labels
fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=600)

<br/><br/><br/>

---


### Extra: fit the data using an even more complicated model!

What if we were really committed to fitting the data? Consider the following model  

$$
\text{SinModel}_{w, \phi}(x) = \sum_{k=1}^{p} w_k sin(2\pi k x + \phi_k)
$$

where $p$ is a user-defined constant which determines the number of sines in the model and the parameters we aim to learn are $\{ w_k, \phi_k \}_{k=1}^p$. 


We can combine this model with our quadratic model to make a new `QuadSinModel` (see below):

$$
\text{QuadSinModel}_{u, w, \phi}(x) = \text{QuadraticModel}_{u}(x) + \text{SinModel}_{w, \phi}(x)
$$



Try changing the number of sine functions `p` and observe how it affects the model.


In [110]:
class SinBasis(nn.Module):
    def __init__(self, p=1):
        super().__init__()
        self.w = nn.Parameter(torch.randn(p,1).double())
        self.phase = nn.Parameter(torch.randn(p,1).double())
        self.freq = torch.tensor(2*np.pi *(1. + np.arange(p))).unsqueeze(0)
        
    def forward(self, x):
        w = self.w
        phase = self.phase
        freq = self.freq
        # Explaining the following calculation:
        #   0. The @ operator is matrix multiplication. 
        #   1. x is a tall (n,1) matrix and freq.T is a row (1,p)-matrix
        #   2. x @ freq => is all combinations of x at each frequency (n, p)-matrix
        #   3. phase.T is "broadcast" (and added) to each row of the (n, p)-matrix
        #   4. torch.sin(x @ freq + phase.T) is an (n, p)-matrix that when multiplied by w
        #      produces an (n, 1) prediction matrix
        predictions = torch.sin(x @ freq + phase.T) @ w
        return predictions

The following model combines our earlier two models to produce a new "better" model.

In [111]:
class QuadSinModel(nn.Module):
    def __init__(self, p=50):
        super().__init__()
        self.quad = QuadraticModel()
        self.sinbasis = SinBasis(p)
        
    def forward(self, x):
        quad_model = self.quad
        sin_model = self.sinbasis
        predictions = quad_model(x) + sin_model(x) 
        return predictions

The following code trains this new better model (and may take a few seconds to run).

In [112]:
# The number of Sine functions to use
p = 200
batch_size = 100 
nepochs = 200 
lr = lambda t: 1./ (1.0 + t) 
dataset = TensorDataset(xs, ys) 

crazy_model = QuadSinModel(p)
sgd(crazy_model, mse_loss, dataset, lr=lr, batch_size=batch_size, nepochs=nepochs)

yhat = crazy_model(xs).detach() 
residual = (ys - yhat).detach().flatten()
print("Loss:", mse_loss(ys, yhat).item())

Loss: 0.004104198708066575


In [113]:
# Visualize predictions
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=xs.flatten(), y=ys.flatten(), name="Data",
                         mode="markers+lines", 
                         marker=go.scatter.Marker(size=2.5),
                         line=go.scatter.Line(width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=xs.flatten(), y=yhat.detach().flatten(), 
                         name='Crazy Model' ),
              row=1, col=1)
# Bottom Plot

fig.add_trace(go.Scatter(x=xs.flatten(), y=residual, name='Residuals'),
              row=2, col=1)
# Axis Labels
fig.update_xaxes(title_text="Year (Standardized)", row=2, col=1)
fig.update_yaxes(title_text="Y", row=1, col=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_layout(height=700)

Feel free to share your clever solutions or improvements to this last step in the Piazza thread [@888](https://piazza.com/class/k4zyqkjkyt33a2?cid=888).

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [114]:
# Save your notebook first, then run this cell to submit.
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... 

ERROR  | auth.py:91 | {'error': 'invalid_grant'}


Saved 'lab05.ipynb'.
Performing authentication
Please enter your bCourses email.
bCourses email: ayelachughtai@berkeley.edu

Copy the following URL and open it in a web browser. To copy,
highlight the URL, right-click, and select "Copy".

https://okpy.org/client/login/

After logging in, copy the code from the web page, paste it below,
and press Enter. To paste, right-click and select "Paste".

Paste your code here: 3kytUZje2dnPgj6A6Uc7PKgyve37DT
Successfully logged in as ayelachughtai@berkeley.edu
Submit... 100% complete
Submission successful for user: ayelachughtai@berkeley.edu
URL: https://okpy.org/cal/data100/sp20/lab05/submissions/Qko057

